<a href="https://colab.research.google.com/github/seamusrobertmurphy/wildfire-extent-global-data/blob/main/Winrock_Ecuador_Wildfire_Delineation_Task_5_060225.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Environment setup**

In [ ]:
!pip install leafmap geemap==0.16.4 geopandas numpy session_info
import ee, json, geemap, ipyleaflet, os, numpy, session_info
from google.colab import drive
from google.colab import files
!drive.mount('/content/drive')

### **Activate Earth Engine**

In [ ]:
!earthengine authenticate
#!ee.Authenticate() # deprecated in certain Colab environments
ee.Initialize(project = "murphys-deforisk")

### **Environment check**

In [3]:
# Check Runtime GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0: print('Not connected to a GPU')
else: print(gpu_info)

# Check Runtime-Allocated RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20: print('Not using a high-RAM runtime')
else: print('You are using a high-RAM runtime!')

Wed Jun 11 21:36:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### **Import AOI: Jurisidiction Boundary**


In [4]:
country = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(
    ee.Filter.equals("ADM0_NAME", "Ecuador"))
state = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Ecuador"))
state_list = country.aggregate_array('ADM1_NAME').distinct().getInfo()
state_list

# derive earthengine centroid for faster rendering
centroid_ee = country.first().geometry().centroid()

### **Map AOI: Jurisdiction Boundary**

In [10]:
red = {"color": "red", "width": 0.5, "lineType": "solid", "fillColor": "00000000"}
white = {"color": "white", "width": 0.5, "lineType": "solid", "fillColor": "00000000"}
country_label = ee.FeatureCollection([ee.Feature(
    country.geometry().centroid(), {'country_name': country.first().get("ADM0_NAME").getInfo()})])

Map = geemap.Map()
Map.centerObject(centroid_ee, 6)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(state.style(**red), {}, "States")
Map.addLayer(country.style(**white), {}, "Country")
Map.add_labels(state,"ADM1_NAME",font_size="6pt",font_color="white",font_family="arial") #,font_weight="bold",)
Map.add_labels(country_label, "country_name", font_size="12pt", font_color="white", font_family="arial",)
Map

Map(center=[-1.4328003721031835, -78.76612016462707], controls=(WidgetControl(options=['position', 'transparen…

### **Import FireCCI51: MODIS Burned Area Dataset**

In [6]:
# Load the FireCCI51 dataset
dataset_reference_period = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate('2012-01-01', '2016-12-31')
dataset_crediting_period = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate('2017-01-01', '2024-12-31')

# Select the BurnDate band
burnedArea_reference_period = dataset_reference_period.select('BurnDate')
burnedArea_crediting_period = dataset_crediting_period.select('BurnDate')

# Use a circular palette to assign colors to date of first detection
baVis = {
  'min': 1,
  'max': 366,
  'palette': [
    'ff0000', 'fd4100', 'fb8200', 'f9c400', 'f2ff00', 'b6ff05',
    '7aff0a', '3eff0f', '02ff15', '00ff55', '00ff99', '00ffdd',
    '00ddff', '0098ff', '0052ff', '0210ff', '3a0dfb', '7209f6',
    'a905f1', 'e102ed', 'ff00cc', 'ff0089', 'ff0047', 'ff0004'
  ]
}

# Get the maximum burn date
maxBA_reference_period = burnedArea_reference_period.max()
maxBA_crediting_period = burnedArea_crediting_period.max()

## **Map Burn: Reference Period**

In [11]:
Map = geemap.Map()
Map.centerObject(centroid_ee, 8)
Map.add_basemap('Esri.WorldImagery')
#Map.addLayer(country.style(**white), {}, "Country")
Map.addLayer(state.style(**white), {}, "States")
Map.addLayer(maxBA_reference_period, baVis, 'Burned Area')
Map

# Get info about the image
#print(maxBA_reference_period.getInfo())

Map(center=[-1.4328003721031835, -78.76612016462707], controls=(WidgetControl(options=['position', 'transparen…

## **Map Burn: Crediting Period**

In [8]:
Map = geemap.Map()
Map.centerObject(centroid_ee, 8)
Map.add_basemap('Esri.WorldImagery')
#Map.addLayer(country.style(**white), {}, "Country")
Map.addLayer(state.style(**white), {}, "States")
Map.addLayer(maxBA_crediting_period, baVis, 'Burned Area')
Map

Map(center=[-1.4328003721031835, -78.76612016462707], controls=(WidgetControl(options=['position', 'transparen…

## **Export Reference & Crediting Period Rasters**

In [ ]:
# Export the maxBA_reference_period raster to Google Drive
task_reference = ee.batch.Export.image.toDrive(
    image=maxBA_reference_period,
    description='maxBA_reference_period',
    folder='Ecuador-Wildfire-Delineation',  # Specify folder in your Google Drive
    fileNamePrefix='maxBA_reference_period',
    region=country.geometry(),
    scale=250,  # Set the desired resolution, we apply native MODIS resolution here before downsampling
    crs='EPSG:4326', # Set the desired coordinate reference system
    maxPixels= 1e13,
    fileFormat= 'GeoTIFF',
    formatOptions= {'cloudOptimized': True}
)

# Start the export task
#task_reference.start()

# Export the maxBA_crediting_period raster to Google Drive
task_crediting = ee.batch.Export.image.toDrive(
    image=maxBA_crediting_period,
    description='maxBA_crediting_period',
    folder='Ecuador-Wildfire-Delineation',  # Specify the same or a different folder
    fileNamePrefix='maxBA_crediting_period',
    region=country.geometry(),
    scale=30,
    crs='EPSG:4326', # Set the desired coordinate reference system
    maxPixels= 1e13,
    fileFormat= 'GeoTIFF',
    formatOptions= {'cloudOptimized': True}
)

# Start the export task
#task_crediting.start()
#print('Export tasks started. Check the Tasks tab in the Earth Engine Code Editor to monitor progress.')

In [ ]:
session_info.show()